In [29]:
# !rm -rf /content/codebase-concept-mapper/

In [30]:
# Cell 1: Setup HuggingFace Token
# Go to: https://huggingface.co/settings/tokens
# Create a token, then add it to Colab secrets

from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')  # Add via left sidebar → 🔑 Secrets
login(token=hf_token)

# Also request access to gated model:
# Visit: https://huggingface.co/google/embeddinggemma-300m
# Click "Request Access" button

In [31]:
!tree -L 2 /content/codebase-concept-mapper -I '__pycache__|*.pyc|.git|test_code|temp_repo'

/content/codebase-concept-mapper
├── DEV
│   ├── Evaluate_Embedding_Models_on_Code_Concepts.md
│   ├── phase1_diagnostic.md
│   ├── phase1_results
│   └── phase1_root_cause_analysis.md
├── docs
│   ├── CONTEXT.md
│   ├── phase1_context.md
│   └── PHASED_PLAN.md
├── phase1
│   ├── all_results.json
│   ├── comparison.md
│   ├── concept_validators.py
│   ├── dataset_generator.py
│   ├── MODEL_RECOMMENDATIONS.md
│   ├── phase1_decision.txt
│   ├── project_structure.md
│   ├── QUICKSTART.md
│   ├── README_PHASE1.md
│   ├── research_query.md
│   ├── results_Alibaba-NLP_gte-multilingual-base.json
│   ├── results_BAAI_bge-small-en-v1.5.json
│   ├── results_google_embeddinggemma-300m.json
│   ├── results_intfloat_multilingual-e5-large-instruct.json
│   ├── results_nomic-ai_CodeRankEmbed.json
│   ├── results_nomic-ai_nomic-embed-text-v1.5.json
│   ├── results_Qwen_Qwen3-Embedding-0.6B.json
│   ├── run_phase1.sh
│   ├── setup_phase1.sh
│   ├── temp_repos
│   ├── test_code_specific_quick.py
│   └─

In [32]:
# Cell 2: Clear GPU Memory
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

print(f"GPU: {torch.cuda.get_device_properties(0).name}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Available Memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

GPU: Tesla T4
Total Memory: 15.83 GB
Available Memory: 15.83 GB


In [33]:
%%capture
# Suppress installation output
!pip install sentence-transformers torch numpy matplotlib

# Mount Google Drive for persistence
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

print("✓ Dependencies installed")
print("✓ Google Drive mounted")

In [34]:
import os
import shutil
from pathlib import Path

# Configuration
REPO_URL = "https://github.com/akbargherbal/codebase-concept-mapper.git"
REPO_NAME = "codebase-concept-mapper"
WORKING_DIR = f"/content/{REPO_NAME}/phase1"

def setup_repository():
    """Clone or sync the repository and navigate to working directory."""

    repo_path = Path(f"/content/{REPO_NAME}")

    # Check if repository already exists
    if repo_path.exists():
        print(f"✓ Repository '{REPO_NAME}' already exists")

        # Check if it's corrupted (common issue)
        try:
            os.chdir(repo_path)
            print("\n→ Syncing with remote...")

            # Fetch latest changes
            !git fetch origin

            # Show current branch and status
            !git branch --show-current
            !git status --short

            # Pull latest changes (assuming main branch, adjust if needed)
            print("\n→ Pulling latest changes...")
            !git pull origin main

        except (FileNotFoundError, OSError) as e:
            print(f"\n⚠️ Repository corrupted: {e}")
            print("🗑️ Removing and re-cloning...")

            # Go to safe directory first
            os.chdir('/content')

            # Force remove corrupted directory
            shutil.rmtree(repo_path, ignore_errors=True)

            # Clone fresh
            print(f"\n→ Cloning repository '{REPO_NAME}'...")
            !git clone {REPO_URL}
            print(f"✓ Repository cloned successfully")

    else:
        print(f"→ Cloning repository '{REPO_NAME}'...")
        os.chdir('/content')  # Make sure we're in a valid directory
        !git clone {REPO_URL}
        print(f"✓ Repository cloned successfully")

    # Verify structure
    print("\n" + "="*50)
    print("REPOSITORY STRUCTURE")
    print("="*50)

    # Check if tree command exists, otherwise use ls
    tree_check = !which tree
    if tree_check:
        !tree -L 2 {repo_path} -I '__pycache__|*.pyc|.git'
    else:
        print("(tree not installed, using ls)")
        !ls -la {repo_path}
        if Path(f"{repo_path}/phase1").exists():
            print(f"\nContents of phase1/:")
            !ls -la {repo_path}/phase1

    # Navigate to working directory
    working_path = Path(WORKING_DIR)
    if working_path.exists():
        os.chdir(WORKING_DIR)
        print(f"\n✓ Changed to working directory: {WORKING_DIR}")
        print(f"Current directory: {os.getcwd()}")

        # Show what's in the working directory
        print("\nFiles in working directory:")
        !ls -lh
    else:
        print(f"\n⚠️ Warning: '{WORKING_DIR}' does not exist")

        # Try to navigate to repo root at least
        try:
            os.chdir(repo_path)
            print(f"Staying in repo root: {os.getcwd()}")
            print("\nAvailable directories:")
            !ls -la
        except:
            print(f"Staying in: /content")
            os.chdir('/content')

# Run setup
setup_repository()

✓ Repository 'codebase-concept-mapper' already exists

→ Syncing with remote...
main
 M phase1/all_results.json
 M phase1/comparison.md
 M phase1/results_nomic-ai_CodeRankEmbed.json
 M phase1/results_nomic-ai_nomic-embed-text-v1.5.json
?? phase1/phase1_decision.txt
?? phase1/results_Alibaba-NLP_gte-multilingual-base.json
?? phase1/results_Qwen_Qwen3-Embedding-0.6B.json
?? phase1/results_google_embeddinggemma-300m.json
?? phase1/results_intfloat_multilingual-e5-large-instruct.json

→ Pulling latest changes...
From https://github.com/akbargherbal/codebase-concept-mapper
 * branch            main       -> FETCH_HEAD
Already up to date.

REPOSITORY STRUCTURE
/content/codebase-concept-mapper
├── DEV
│   ├── Evaluate_Embedding_Models_on_Code_Concepts.md
│   ├── phase1_diagnostic.md
│   ├── phase1_results
│   └── phase1_root_cause_analysis.md
├── docs
│   ├── CONTEXT.md
│   ├── phase1_context.md
│   └── PHASED_PLAN.md
├── phase1
│   ├── all_results.json
│   ├── comparison.md
│   ├── concept_v

In [35]:
# Silent install (suppress output)
!pip install -r /content/codebase-concept-mapper/requirements.txt -q

print("✓ Dependencies installed")

✓ Dependencies installed


In [36]:
!python /content/codebase-concept-mapper/phase1/test_code_specific_quick.py

2025-12-04 08:44:24.597244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764837864.616914    4763 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764837864.623045    4763 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764837864.638962    4763 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764837864.638987    4763 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764837864.638991    4763 computation_placer.cc:177] computation placer alr

In [37]:
import torch

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected!")
    print("   Go to: Runtime > Change runtime type > Select T4 GPU")

✅ GPU: Tesla T4
   Memory: 15.8 GB


In [38]:
from google.colab import drive

drive.mount('/content/drive')
print("✓ Google Drive mounted")
print("   Backups will be saved to: /content/drive/MyDrive/phase1_results_*")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted
   Backups will be saved to: /content/drive/MyDrive/phase1_results_*


In [39]:
%%time
# Generate with Drive backup
!python dataset_generator.py

# Verify
import json
from pathlib import Path

if Path("test_code/metadata.json").exists():
    with open("test_code/metadata.json") as f:
        meta = json.load(f)
    print(f"\n✅ Dataset ready!")
    print(f"   Total: {meta['stats']['total_files']} files")
    print(f"   Python: {meta['stats']['python_files']}")
    print(f"   JavaScript: {meta['stats']['javascript_files']}")
else:
    print("❌ Generation failed - check errors above")


PHASE 1: GENERATING TEST DATASET (COLAB OPTIMIZED)
✓ Created output directories in test_code

📦 Processing PYTHON repositories...

  Repository: pallets/flask
  Repository already exists at temp_repos/flask
  Found 12 candidate files
  Sampled 12 files

  Repository: psf/requests
  Repository already exists at temp_repos/requests
  Found 9 candidate files
  Sampled 9 files

  Repository: aio-libs/aiohttp
  Repository already exists at temp_repos/aiohttp
  Found 28 candidate files
  Sampled 12 files

  Repository: django/django
  Repository already exists at temp_repos/django
  Found 353 candidate files
  Sampled 12 files

  ✓ PYTHON: 45 files collected

📦 Processing JAVASCRIPT repositories...

  Repository: axios/axios
  Repository already exists at temp_repos/axios
  Found 24 candidate files
  Sampled 12 files

  Repository: expressjs/express
  Repository already exists at temp_repos/express
  Found 3 candidate files
  Sampled 3 files

  Repository: facebook/react
  Repository alread

In [40]:
%%time
# Test candidate models
!python test_embeddings.py

2025-12-04 08:44:49.839162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764837889.858951    4941 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764837889.864888    4941 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764837889.879607    4941 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764837889.879632    4941 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764837889.879636    4941 computation_placer.cc:177] computation placer alr

In [41]:
# Display comparison table
from IPython.display import Markdown, display

with open("comparison.md", "r") as f:
    display(Markdown(f.read()))


## Model Comparison

| Model | P@5 | P@1 | MRR | Pass Rate | Time (s) | Device | Decision |
|-------|-----|-----|-----|-----------|----------|--------|----------|
| Alibaba-NLP/gte-multilingual-base | 25.0% | 30.0% | 0.458 | 20.0% | 4.8 | cuda | ❌ NO-GO |
| nomic-ai/nomic-embed-text-v1.5 | 24.0% | 20.0% | 0.403 | 15.0% | 5.2 | cuda | ❌ NO-GO |
| Qwen/Qwen3-Embedding-0.6B | 23.0% | 40.0% | 0.525 | 10.0% | 17.9 | cuda | ❌ NO-GO |
| intfloat/multilingual-e5-large-instruct | 23.0% | 20.0% | 0.318 | 15.0% | 2.4 | cuda | ❌ NO-GO |
| google/embeddinggemma-300m | 7.0% | 5.0% | 0.104 | 5.0% | 7.9 | cuda | ❌ NO-GO |

### Recommendation

**❌ STOP PROJECT** - Embedding approach not viable

- Best model: `Alibaba-NLP/gte-multilingual-base` at 25.0% P@5
- Threshold: ≥50% for conditional proceed
- Recommendation: Consider alternative approaches or wait for better models


In [42]:
import json

with open("all_results.json") as f:
    data = json.load(f)

models = [m for m in data["all_models"] if "error" not in m]
best = max(models, key=lambda x: x["overall_precision_at_5"])

print(f"🏆 Best Model: {best['model']}")
print(f"   P@5: {best['overall_precision_at_5']:.1%}")
print(f"   P@1: {best['overall_precision_at_1']:.1%}")
print(f"   Device: {best['device']}")

# Failed concepts
failed = [c for c in best['per_concept'] if c['precision_at_5'] < 0.6]
if failed:
    print(f"\n❌ Failed Concepts ({len(failed)}):")
    for c in failed[:5]:
        print(f"   - {c['concept']}")

# Top concepts
top = sorted(best['per_concept'], key=lambda x: x['precision_at_5'], reverse=True)[:5]
print(f"\n✅ Top Concepts:")
for c in top:
    print(f"   - {c['concept']}: {c['precision_at_5']:.2f}")

🏆 Best Model: Alibaba-NLP/gte-multilingual-base
   P@5: 25.0%
   P@1: 30.0%
   Device: cuda

❌ Failed Concepts (16):
   - context managers python
   - decorators python
   - list comprehensions python
   - generators python
   - file handling python

✅ Top Concepts:
   - class inheritance python: 0.80
   - async await python: 0.60
   - exception handling python: 0.60
   - array methods javascript: 0.60
   - react hooks: 0.40


In [43]:
import shutil
from pathlib import Path
from datetime import datetime

# Create timestamped backup
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_dir = Path(f"/content/drive/MyDrive/phase1_results_{timestamp}")
backup_dir.mkdir(parents=True, exist_ok=True)

# Copy results
files_to_backup = [
    "all_results.json",
    "comparison.md",
    "test_code/metadata.json"
]

for f in files_to_backup:
    if Path(f).exists():
        shutil.copy(f, backup_dir / Path(f).name)

# Copy per-model results
for f in Path(".").glob("results_*.json"):
    shutil.copy(f, backup_dir / f.name)

print(f"✅ Backed up to: {backup_dir}")
print(f"\n📂 Files saved:")
for f in backup_dir.iterdir():
    print(f"   - {f.name}")

✅ Backed up to: /content/drive/MyDrive/phase1_results_20251204_084619

📂 Files saved:
   - all_results.json
   - comparison.md
   - metadata.json
   - results_intfloat_multilingual-e5-large-instruct.json
   - results_Alibaba-NLP_gte-multilingual-base.json
   - results_nomic-ai_CodeRankEmbed.json
   - results_BAAI_bge-small-en-v1.5.json
   - results_Qwen_Qwen3-Embedding-0.6B.json
   - results_nomic-ai_nomic-embed-text-v1.5.json
   - results_google_embeddinggemma-300m.json


In [44]:
# Save decision for Phase 2
with open("phase1_decision.txt", "w") as f:
    f.write(f"DECISION: GO TO PHASE 2\n")
    f.write(f"Model: {best['model']}\n")
    f.write(f"P@5: {best['overall_precision_at_5']:.1%}\n")

print("✅ Decision saved to phase1_decision.txt")

✅ Decision saved to phase1_decision.txt
